In [ ]:
import os, pathlib, sys
REPO_ROOT = pathlib.Path.cwd().parent
os.chdir(REPO_ROOT)           # move from notebooks/ to repo root
sys.path.insert(0, str(REPO_ROOT))

In [ ]:
import numpy as np, pandas as pd
from src.models.knn_recommender import recommend

In [ ]:
seed = 2
rec_ids, dists = recommend(seed, k=10)

# Structural checks
assert len(rec_ids) == 10
assert len(set(rec_ids)) == 10             # no duplicates
assert seed not in rec_ids                 # self not included
assert all(dists[i] <= dists[i+1] for i in range(len(dists)-1))

# Inspect metadata
idx = pd.read_parquet("data/processed/fma_small_index.parquet")
meta = idx[idx.track_id.isin(rec_ids)][
    ["track_id","artist_name","track_title","album_title","track_genre_top"]
].copy()
order = {tid:i for i,tid in enumerate(rec_ids)}
meta["order"] = meta["track_id"].map(order)
meta = meta.sort_values("order").drop(columns="order")
print(meta.to_string(index=False))

# Genre distribution sanity
print("\nGenre counts:\n", meta["track_genre_top"].value_counts())

# Optional: convert distances to cosine similarity for intuition
cos_sim = [1 - d for d in dists]
print("\nCosine similarities:", np.round(cos_sim, 3))